# Pymol In GCP

## Overview


## Setup
To run PyMOL in GCP we are going to run pymol within a container on this Vertex AI instance. Unfortunately with how PyMOL works we are unable to run PyMOL commands from within the jupyter notebook cells or the instances terminal. However, we are able to use the PyMOL GUI, PyMOL command line, or write PML scripts in the notebook and then run them within PyMOL. The instructions to get PyMOL started are at the end of the notebook.


## Common PyMOL Commands
| Loading and Saving | Visualization | Selection and Manipulation | View Control | Appearance | Analysis |
|-------------------|---------------|---------------------------|--------------|------------|-----------|
| `load`: Loads structures | `show`: Controls display | `select`: Creates selections | `zoom`: Zooms to selection | `color`: Changes colors | `align`: Aligns structures |
| `fetch`: Downloads from PDB | `hide`: Hides representations | `delete`: Removes objects | `center`: Centers view | `label`: Adds labels | `set`: Changes settings |
| `save`: Saves session/structure | `cartoon`: Shows 2° structure | `bond`: Creates bonds | `rotate`: Rotates view | `ray`: Creates HD images | |
| | `surface`: Shows surface | `split_states`: Separates states | `clip`: Adjusts clipping | | |

## Writing PML files
Each line on a pml file is a PyMOL command. 

Example.pml: <br>
fetch 3bcq <br>
set seq_view,1<br>
util.color_chains()


Using Python we can write a pml file which is able to be accessed by PyMOL. When writting the file it is important to make sure that each command is on its own line. So for example the first line of the script the PyMOL command is fetch 3bcq so when writting to the file in Python we need to add the next line token "\n". Bellow is the code to write the Example.pml script.

In [1]:
# Write the pml file
with open("script.pml","w") as scriptout:
    scriptout.write("fetch 3bcq\n") # Loads protein
    scriptout.write("set seq_view,1\n") # colors protein
    scriptout.write("util.color_chains()\n") # colors protein


1a.  Select file run script and then the pml script within pymol 

1.b Alternatively, you can set your directory into the folder and then run:
pymol> @script.pml

<div class="alert alert-block alert-warning"> <b>Attention:</b> Throughout the notebooks we will be giving the PyMOL commands by writting the pml file. It is encouraged that when trying to learn instead of running the pml script you take the time to run each of the PyMOL commands using the PyMOL command line and understand why each command is run and what is happening.</a>. </div>

## Commands To Start PyMOL

In [2]:
import subprocess
import urllib.request
import sys
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
import json
import subprocess
subprocess.run('pip install google-cloud-compute notebook'.split(" "), text=True, capture_output=True)
import urllib.request
from __future__ import annotations

import sys
from typing import Any

from google.api_core.extended_operation import ExtendedOperation
from google.cloud import compute_v1

###  Run the below cell and copy its output to a new cell and run it

In [4]:
%%javascript
// Send data

fetch( 'https://api64.ipify.org/')
  .then(
    response => response.text()
  ).then(
    text => element.append("ip='"+text+"'")
  );

<IPython.core.display.Javascript object>

In [6]:
ip='2401:4900:1c6a:f559:85ec:4dc1:405b:c3a3'

<div class="alert alert-block alert-danger"> <b>Attention:</b> Make sure to assign the returned ip address to the ip variable.</a>. </div> <br>

### Run the below to create a firewall rule to allow access to the GUI

In [7]:
firewall_rule_capture = subprocess.run('gcloud compute firewall-rules list'.split(" "), text=True, capture_output=True) 
rules = firewall_rule_capture.stdout.split("\n")
rule_found = False
for x in rules:
    if 'allow-gui-accesssr1' in x:
        rule_found = True
        print("rule already exist")
if not rule_found:
    subprocess.run(['gcloud','compute','firewall-rules','create','allow-gui-accesssr1','--allow=tcp:8081','--source-ranges='+ip+'/32','--description="allow-gui-access"'])

rule already exist


### Finally run this block

In [8]:
url = "http://metadata.google.internal/computeMetadata/v1/instance/name"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
machine_name = urllib.request.urlopen(req).read().decode()
url = "http://metadata.google.internal/computeMetadata/v1/instance/zone"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
zone = urllib.request.urlopen(req).read().decode().split("/")[-1]
url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
project_id = urllib.request.urlopen(req).read().decode()
instance_client = compute_v1.InstancesClient()
i = instance_client.get(project=project_id, zone=zone, instance=machine_name)
ip = i.network_interfaces[0].access_configs[0].nat_i_p

# Directly launch PyMOL Docker container
docker_out = subprocess.Popen(
    ('docker run --rm -v /home/jupyter:/config/s3 -p 8081:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/pymol').split(" ")
)


display(HTML(f"Please access PyMOL at <a href=\"http://{ip}:8081\">http://{ip}:8081</a><br>Note, it may take 30 seconds for it to fully start"))

[migrations] started
[migrations] no migrations found
usermod: no changes
───────────────────────────────────────
  _____ __ __ _____ _____ _____ _____ 
 |     |  |  |   __|_   _|     |     |
 |   --|  |  |__   | | | |  |  | | | |
 |_____|_____|_____| |_| |_____|_|_|_|
       _____ __ __ _ __    ____  
      | __  |  |  | |  |  |    \ 
      | __ -|  |  | |  |__|  |  |
      |_____|_____|_|_____|____/ 

  Based on images from linuxserver.io
───────────────────────────────────────

To support LSIO projects visit:
https://www.linuxserver.io/donate/

───────────────────────────────────────
GID/UID
───────────────────────────────────────

User UID:    911
User GID:    911
───────────────────────────────────────



.+.....+....+..+.......+.....+.+..+............+.+.....+.+........+......+..........+...........+...............+......+...+.+.....+.+........+....+...+.........+..............+....+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+...............+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..+.........+.+...+..+...+.......+.................+.......+........+...+...+.+.........+.....+.......+...............+..............+.+...+..+..........+..+................+.........+......+..+....+..............+.+......+..+......+.+...+...+..+...+......................+..+..........+...+..+...+..........+...+..+......................+......+..+.+..+....+...+...+.....+.......+........+...+.......+.........+..............+....+.....+......+.+...+..+.+..............+....+.........+..+...+...+.............+..............+...+.........+...+.+......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
..++++++++++++++++++++++++++++++++++++++++++++

[custom-init] No custom files found, skipping...
[ls.io-init] done.


_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.

Xvnc KasmVNC 1.3.2 - built Oct 26 2024 19:02:31
Copyright (C) 1999-2018 KasmVNC Team and many others (see README.me)
See http://kasmweb.com for information on KasmVNC.
Underlying X server release 12014000, The X.Org Foundation



Obt-Message: Xinerama extension is not present on the server
19


QStandardPaths: wrong permissions on runtime directory /config/.XDG, 0755 instead of 0700


 PyMOL(TM) Molecular Graphics System, Version 2.5.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Detected OpenGL version 4.5. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.50.
 OpenGL graphics engine:
  GL_VENDOR:   Mesa
  GL_RENDERER: llvmpipe (LLVM 15.0.7, 256 bits)
  GL_VERSION:  4.5 (Compatibility Profile) Mesa 23.2.